In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.applications import DenseNet121

# Load DeepLabV3+ from TensorFlow Hub
model = tf.keras.applications.DenseNet121(weights="imagenet", include_top=False)


def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (512, 512))  # Resize to DeepLabV3+ input size
    image = image / 255.0  # Normalize
    return np.expand_dims(image, axis=0)


def segment_leaf(image_path):
    input_tensor = preprocess_image(image_path)
    prediction = model.predict(input_tensor)
    mask = np.argmax(prediction[0], axis=-1)  # Get class-wise segmentation
    return mask


# Test with an image
image_path = "leaf.jpg"
mask = segment_leaf(image_path)
cv2.imwrite("leaf_mask.jpg", mask * 255)  # Save mask

2025-01-31 10:40:58.381104: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738320058.474608   29159 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738320058.506024   29159 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
I0000 00:00:1738320063.436648   29159 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3586 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 38s 1us/step


I0000 00:00:1738320109.599780   29803 service.cc:148] XLA service 0x7f4540003f80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1738320109.600186   29803 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
I0000 00:00:1738320110.694186   29803 cuda_dnn.cc:529] Loaded cuDNN version 90300


1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step


I0000 00:00:1738320121.044192   29803 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
[ WARN:0@58.713] global loadsave.cpp:848 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


True

In [4]:
print("Unique values in mask:", np.unique(mask))

Unique values in mask: [ 138  409  534  548  555  567  572  578  585  654  672  674  703  709
  712  724  740  756  778  784  796  810  819  822  855  893  907  914
  926  948  949  996 1009 1011]


In [2]:
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Conv2DTranspose,
    Concatenate,
    Input,
)
from tensorflow.keras.models import Model


def unet_model(input_shape=(256, 256, 3)):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, (3, 3), activation="relu", padding="same")(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), activation="relu", padding="same")(pool1)
    pool2 = MaxPooling2D((2, 2))(conv2)

    # Bottleneck
    conv3 = Conv2D(256, (3, 3), activation="relu", padding="same")(pool2)

    # Decoder
    up1 = Conv2DTranspose(128, (3, 3), strides=(2, 2), padding="same")(conv3)
    concat1 = Concatenate()([up1, conv2])

    up2 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(concat1)
    concat2 = Concatenate()([up2, conv1])

    outputs = Conv2D(1, (1, 1), activation="sigmoid")(concat2)

    return Model(inputs, outputs)


model = unet_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │      1,792 │ input_layer_1[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 128, 128,  │    295,040 │ conv2d_2[0][0]    │
│ (Conv2DTranspose)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 128, 128,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 256)              │            │ conv2d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 256, 256,  │    147,520 │ concatenate[0][0] │
│ (Conv2DTranspose)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 256, 256,  │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 128)              │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 256, 256,  │        129 │ concatenate_1[0]… │
│                     │ 1)                │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 813,505 (3.10 MB)

 Trainable params: 813,505 (3.10 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Load dataset
X_train = np.load("X_train.npy")  # Leaf images
Y_train = np.load("Y_train.npy")  # Segmentation masks (from DeepLabV3+)

# Train model
model.fit(X_train, Y_train, epochs=10, batch_size=16, validation_split=0.2)
model.save("leaf_segmentation_unet.keras")  # Save trained U-Net

FileNotFoundError: [Errno 2] No such file or directory: 'X_train.npy'

In [ ]:
def predict_leaf(image_path, model_path="leaf_segmentation_unet.h5"):
    model = tf.keras.models.load_model(model_path)
    image = preprocess_image(image_path)
    mask = model.predict(image)[0]  # Get segmentation mask
    return mask


image_path = "new_leaf.jpg"
leaf_mask = predict_leaf(image_path)
cv2.imwrite("final_leaf_mask.jpg", leaf_mask * 255)  # Save refined mask